# Técnicas de Análisis

## Carlos Ramón Santonja
Este notebook corresponde con la práctica 2 de la asignatura Minería de Datos, dentro del Máster Ciencia de Datos de la Universidad de Alicante. Se divide en dos partes. 
1. Análisis de Regresión y clasificación
1. Ensembles, clustering y reglas de asociación






El primer paso, común en ambas partes  consiste en cargar las librerías que permiten el tratamiento de los datos y la visualización. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from collections import Counter

## Parte 1 
### Regresión

Para el problema de regresión se han usado unos datos relacionados con la energía solar, proporcionados por la nasa. Este conjunto de datos presenta 11 columnas que contienen información sobre la fecha y diferentes propiedades físicas (radiación, temperatura, presión, humedad, etc.). El conjunto de datos presenta 32686 filas. 

El objetivo del análisis es aplicar modelos de regresión para tratar de predecir la temperatura a partir de los datos temporales, de la radiación, de la presión, de la humedad y de la velocidad del viento. 

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
data = pd.read_csv('../input/SolarEnergy/SolarPrediction.csv')
data

Estudiemos un poco el conjunto de datos, para tratar de entender los valores de cada variable. 

In [ ]:
data.describe()

Convirtamos las columnas de fecha al formato de pandas, para que resulte más fácil trabajar con esta información. Vemos que las fechas aparecen en dos formatos distintos, un formato Unix y otro en formato fecha y hora, en dos columnas separadas. Podemos convertir estas dos columnas a una única columna de tipo datetime de pandas, quedándonos únicamente con una columnas de fecha. 

In [ ]:
data['data_time'] = pd.to_datetime(data['Data'].str.split(' ', expand=True).iloc[:,0] + ' '+ data['Time'])
data.drop(columns=['UNIXTime', 'Data', 'Time'], inplace=True)

Por comodidad, reorganizamos las columnas para que la primera columna que aparezca sea la de fecha. Además, renombramos los nombres de las columnas para que aparezcan siempre en minúsculas.

In [ ]:
data = data[['data_time'] + data.columns.tolist()[:-1]]
data.columns = data.columns.str.lower()
data.rename(columns={'data_time':'time', 'winddirection(degrees)':'direction'}, inplace=True)
data

A continuación se muestran algunas gráficas que muestran la evolución temporal de la radiación, temperatura, presión, humedad y velocidad del viento de os últimos días que se tienen registros.

In [ ]:
aux = data[data['time'] > pd.to_datetime('2016-12-22')]
fig, axes = plt.subplots(5,1, figsize=(20,30))

axes[0].plot(aux['time'], aux['radiation'])
axes[0].set_title('Radiación')

axes[1].plot(aux['time'], aux['temperature'])
axes[1].set_title('Temperatura')

axes[2].plot(aux['time'], aux['pressure'])
axes[2].set_title('Presión')

axes[3].plot(aux['time'], aux['humidity'])
axes[3].set_title('Humedad')

axes[4].plot(aux['time'], aux['speed'])
axes[4].set_title('Velocidad del viento')

plt.show()

 Se aprecia una periodicidad diaria, muy notable sobre todo en los datos de radiación y temperatura. La velocidad del viento parece que no influye para nada en la temperatura, ya que varía mucho de un momento a otro, y no parece que exista ninguna relación de causalidad entre este parámetro y la temperatura. Por lo tanto, no vamos a incluir esta característica en nuestro análisis. De la radiación si que parezca que exista una relación causal con la temperatura, ya que los máximos de temperatura coinciden aproximadamente con los máximos de temperatura. De la presión y humedad es dificil sacar alguna conclusión de manera tan precipitada. 

Ahora representamos la relación entre la temperatura y la radiación.

In [ ]:
plt.plot(data['temperature'], data['radiation'], 'o')
plt.xlabel('Temperatura')
plt.ylabel('Radiación')
plt.title('Relación entre la temperatura y la radiación')
plt.show()

A continuación se representa la relación entre la temperatura y la humedad. 

In [ ]:
plt.plot(data['temperature'], data['humidity'], 'o')
plt.xlabel('Temperatura')
plt.ylabel('Humedad')
plt.title('Relación entre la temperatura y la humedad')
plt.show()

Por último graficamos la relación entre la temperatura y la presión. 

In [ ]:
plt.plot(data['temperature'], data['pressure'], 'o')
plt.xlabel('Temperatura')
plt.ylabel('Presión')
plt.title('Relación entre la temperatura y la presión')
plt.show()

A continuación procedemos con el análisis de regresión. El primer modelo realizado ha sido un modelo de regresión lineal. Se prodría escribir como
$$ \text{Temperatura}= \beta_0 + \beta_1 \text{Radiacion}  + \beta_2 \text{Humedad} + \beta_3 \text{Presion}$$
donde los coeficientes $\beta_i$ son los términos a determinar por el análisis. Para seleccionar el modelo vamos a utilizar un cross validation (validación cruzada) de 5 iteraciones. 

In [ ]:
X = data[['radiation', 'pressure', 'humidity']]
y = data['temperature']

cv = KFold(n_splits=5)
R2_s = []

for train_index, test_index in cv.split(data):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]
    model = LinearRegression().fit(X_train,y_train)
    R2_s.append(model.score(X_test,y_test))

print('Valores de R2 para cada modelo: ', *R2_s, sep='\n\t∘ ')
print('\nValor medio de R2: \n\t∘ ', np.mean(R2_s))

Para cada modelo obtenemos un valor de $R2$ distintos y alejados del valor perfecto 1. Esto significa que los datos no siguen una relación lineal y que el modelo no sirve para nada. Como tenemos muchos datos, tratemos de crear un modelo de regresión KNN. De nuevo utilizamos una validación cruzada con 7 iteraciones. 

In [ ]:


R2_s = []
cv = KFold(n_splits=7)

for train_index, test_index in cv.split(data):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]

    model = KNeighborsRegressor()
    model.fit(X_train, y_train)
    R2_s.append(model.score(X_test, y_test))

print('Valores de R2 para cada modelo: ', *R2_s, sep='\n\t∘ ')
print('\nValor medio de R2: \n\t∘ ', np.mean(R2_s))

En este modelo, los valores están algo más cerca de 1 que en el modelo anterior, pero los resultados siguen siendo catastróficos. Para cada selección de datos, el modelo predice unos valores de $R^2$ completamente distintos. Este modelo de regresión tampoco se ajusta correctamente a los datos. 

Por último, probemos con otro modelo de regresión, el random forest regressor. 

In [ ]:
R2_s = []
cv = KFold(n_splits=7)

for train_index, test_index in cv.split(data):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]

    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    R2_s.append(model.score(X_test, y_test))

print('Valores de R2 para cada modelo: ', *R2_s, sep='\n\t∘ ')
print('\nValor medio de R2: \n\t∘ ', np.mean(R2_s))

En este caso, los valores de $R^2$ obtenidos son similares a los obtenidos con los otros modelos, ya que para cada selección de datos se obtiene un valor distinto.

Por tanto, concluimos que con los datos disponibles no tenemos suficiente información para predecir correctamente la temperatura, ya que este parámetro dependerá de muchos más regresores que no estamos considerando. 


### Clasificación

Para el algorito de clasificación, se ha utilizado el dataset `star-dataset`, que se encuentra en la página web a https://www.kaggle.com/deepu1109/star-dataset. Este dataset cuenta con 240 filas y siete columnas, que son: 
* Temperatura
* Luminosidad
* Radio
* Magnitud absoluta
* Tipo estelar
* Color de la estrella
* Clase espectral

El objetivo de este análisis va a ser construir un modelo que sea capaz de predecir su clase espectral en función de la temperatura, el radio, la magnitud absoluta y el tipo estelar. La clase espectral de una estrella se define como una de las siguientes letras: O, B, A, F, G, K, M y está relacionado con las características espectrales que presente la estrella 

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_confusion_matrix, accuracy_score

In [ ]:
stars = pd.read_csv('../input/star-dataset/6 class csv.csv')
stars

In [ ]:
#Renombramos las columnas por comodidad
stars.rename({
    'Temperature (K)': 'Temperatura',
    'Luminosity(L/Lo)': 'Luminosidad',
    'Radius(R/Ro)': 'Radio',
    'Absolute magnitude(Mv)': 'Magnitud',
    'Star type': 'type',
    'Star color': 'color',
    'Spectral Class': 'class'
}, axis=1, inplace=True)

Estudiemos como se relacionan las variables numéricas con la clase estelar.

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(16,9))
for i,ax in enumerate(axes.flatten()):
    sns.scatterplot(data=stars, x=stars.columns[i], y='class', hue='class', ax=ax)


Así parece complicado ver relaciones entre las variables numéricas y la clase estelar a la que pertenece cada estrella.

Veamos si podemos apreciar alguna relación entre la clase estelar y el tipo de la estrella. El tipo de la estrella es una clasificación que asigna a cada estrella un valor entre 0 y 5, por tanto es una variable categórica. 


In [ ]:
sns.scatterplot(data=stars, x='type', y='class', hue='class')

Veamos lo balanceados o desbalanceados que están nuestros datos. Para ello graficamos el histograma de las clases estelares.  

In [ ]:
hist = sns.histplot(data=stars, x='class')
hist.set_title('Clases estelares en el dataset')

Vemos que las clases están desbalaneadas, ya que en nuestro conjunto de datos hay muchas más estrellas de clase M que de otra clase. Este hecho puede suponer un problema a la hora de entrenar el modelo, ya que si  el modelo podría aprender muchas más clases M y fallar a la hora de hacer predicciones. Para solventar este problema, la mejor forma sería añadiendo más datos de las otras clases. Podríamos buscar más datasets que contengan los mismos tipos de datos o incluso duplicar datos de las clases desbalanceadas. En este caso, como tenemos muy pocos datos de las clases G y K, vamos a directamente eliminar estas filas, así nuestro modelo tendrá que predecir menos clases estelares.  

In [ ]:
reg_name = ['Temperatura', 'Luminosidad','Radio', 'Magnitud']
class_name = 'class'
stars = stars[(stars[class_name] != 'G') & (stars[class_name] != 'K')].reset_index()
hist = sns.histplot(data=stars, x='class')
hist.set_title('Clases estelares en el dataset (sin las clases G y K)')

Los datos siguen estando desbalanceados, habiendo bastantes más estrellas de clase M que de las demás. Veamos si con estos datos, los algorimtos de clasificación pueden obtener buenos resultados. 

A continuación se crea el modelo. Se va a utilizar un algoritmo de clasificación SVC (Support Vector Classification). La principal ventaja de este algoritmo de clasificación es que puede devolver las probabilidades. El propio modelo devuelve cuál es la clase más probable a la que pertenece una estrella, junto a la probabilidad de que la estrella pertenezca a cada clase. 

Los datos se van a dividir en dos conjuntos, uno de entrenamiento y uno de validación. El de entrenamiento va a tener el 70% de los casos, y el de validación el 30% restante.


In [ ]:
train, test = train_test_split(stars, test_size=0.3)

model = SVC(probability=True)
model.fit(train[reg_name], train[class_name])


Veamos la validez de este método. Para ello, primero imprimimos las métricas calculadas con los mismos datos utilizados para el entrenamiento. 

In [ ]:
y_true = train[class_name].to_list()
y_pred = model.predict(train[reg_name])
accuracy = accuracy_score(y_true, y_pred)
print(classification_report(y_true, y_pred, zero_division=0))
print('accuracy score -> {0:.2%}'.format(accuracy))

Vemos que el modelo predice fatal,  y eso que estamos validando con los datos usados para el entrenamiento. Conseguimos una accuracy del 52%. Esto significa que el modelo predice la clase a la que pertenece una estrella correctamente la mitad de las veces. Estudiando el f1-score de cada clase vemos que el modelo solo está prediciendo las estrellas que pertenecen a clase M y O. Y aún así, para estas clases presenta unos f1-scores bastante bajos, ya que el valor máximo de esta magnitud es 1. 

Veamos qué pasa si utilizamos datos que el modelo no ha visto en el proceso de entrenamiento. 

In [ ]:
y_true = test[class_name].to_list()
y_pred = model.predict(test[reg_name])
accuracy = accuracy_score(y_true, y_pred)
print(classification_report(y_true, y_pred, zero_division=0))
print('accuracy score -> {0:.2%}'.format(accuracy))

En este caso los resultados son bastante similares, la accuracy sigue siendo bastante baja, del 60%. Vemos que el modelo solo ha aprendido a predecir los datos de las clases M y O, ya que las demás clases obtienen un F1-score de 0. Esto puede ser debido a que los datos de entrenamiento estaban desbalanceados. 

In [ ]:
plot_confusion_matrix(model, test[reg_name], test[class_name])

Vemos que la matriz de confusión presenta 0 en todas las clases que el modelo no es capaz de predecir. En las clases que sí es capaz de predecir M y O, lo hace bastant mal, que predice 32 clases M correctamente y solamente 6 clases O bien. 

Veamos si balanceando las clases podemos conseguir un modelo que funcione mejor. Se van a eliminar algunas filas de las estrellas de clase M en los datos para este fin. El problema de esta operación es que perdemos información y puede que el modelo no tenga suficientes datos para entrenar correctamente. 

In [ ]:
stars_adj = stars[stars[class_name] != 'M']
stars_adj = stars_adj.append(stars[stars[class_name] == 'M'].iloc[0:50]).reset_index()
print(Counter(stars_adj[class_name]))
hist = sns.histplot(data=stars_adj, x='class')
hist.set_title('Clases estelares en el dataset balanceadas')
plt.show()

In [ ]:
train, test = train_test_split(stars_adj, test_size=0.3)
model = SVC(probability=True)
model.fit(train[reg_name], train[class_name])

print(30*'-' + 'TRAIN REPORT MODEL' + 30*'-' + '\n')
y_true = train[class_name].to_list()
y_pred = model.predict(train[reg_name])
accuracy = accuracy_score(y_true, y_pred)
print(classification_report(y_true, y_pred, zero_division=0))
print('accuracy score -> {0:.2%}\n\n'.format(accuracy))

print(30*'-' + 'TEST REPORT MODEL' + 30*'-' + '\n')
y_true = test[class_name].to_list()
y_pred = model.predict(test[reg_name])
accuracy = accuracy_score(y_true, y_pred)
print(classification_report(y_true, y_pred, zero_division=0))
print('accuracy score -> {0:.2%}'.format(accuracy))


Con estos datos más balanceados, obtenemos un resultado bastante similar al anterior. La accuracy calculada con los datos de entrenamiento es de un 48%, bastante baja para ser con los datos con los que se ha entrenado. Con los datos de test es aún más baja, del 34%. Vemos que este modelo solo es capaz de predecir dos clases, B y O, y con un F1-score bastante bajo. Vemos que el eliminar algunos datos no ha aportado mayor precisión. Para mejorar este modelo necesitaríamos un conjunto de datos más elevado, ya que el modelo no es capaz de aprender lo suficiente. Grafiquemos ahora la matriz de confusión. 

In [ ]:
plot_confusion_matrix(model, stars[reg_name], stars[class_name])
plt.show()

Vemos que esta matriz no es para nada diagonal. Presenta múltiples ceros en las diagonales debido a que no se han predicho todas las clases. 

Concluimos que este modelo de clasificación es pésimo. Esto era de esperar ya que el conjunto de datos seleccionados esán desbalanceados y presentan muy pocas líneas. Por esta última razón, el modelo no es capaz de aprender a predecir correctamente a qué clase pertenece cada estrella. Para mejorar este modelo habría que conseguir un dataset con muchos más datos balanceados.

## Parte 2: Ensembles, clustering y reglas de asociación
### Ensembles


A continuación vamos a realizar un algoritmo de clasificación basado en ensembles. Igual que en el apartado anterior, se van a clasificar clases estelares en función de todas las demás variables. 

Vamos a construir un Bagging (*Bootstrap
aggregating*) utilizando un clasificador de partida de *Decission Tree Classifier*. 

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

num_models=10
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=2, max_leaf_nodes=2),
                            n_estimators=10,
                            max_samples=0.5, 
                            max_features=0.5)


Entrenamos el modelo con los datos desbalanceados `stars`.

In [ ]:
X = stars[reg_name].to_numpy()
y = stars[class_name].to_numpy()

bagging.fit(X, y)

In [ ]:
y_pred = bagging.predict(X)
print(classification_report(y, y_pred, zero_division=0))
print('accuracy score -> {0:.2%}\n\n'.format(accuracy_score(y, y_pred)))


En este caso, el algoritmo funciona algo mejor que con simplemente el modelo de clasificación, ya que obtiene una accuracy de casi el 75%. Sin embargo, tampoco es capaz de predecir todas las clases, ya que no predice ninguna estrella de clase A y F. Vemos que las estrellas de clase M las predice muy bien, con un F1-score del 99%. Grafiquemos ahora la matriz de confusión. 

In [ ]:
plot_confusion_matrix(bagging, stars[reg_name], stars[class_name])

En este caso, la matriz de confusión si se parece a una matriz diagonal. Vemos que la clase M se predice muy bien, mientras que las clases B y O se predicen con algunos fallos. 

A continuación se repite este modelo, pero ahora utilizando los datos balanceados.

In [ ]:
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=2, max_leaf_nodes=2),
                            n_estimators=10,
                            max_samples=0.5, 
                            max_features=0.5)

X = stars_adj[reg_name].to_numpy()
y = stars_adj[class_name].to_numpy()

bagging.fit(X, y)

y_pred = bagging.predict(X)
print(classification_report(y, y_pred, zero_division=0))
print('accuracy score -> {0:.2%}\n\n'.format(accuracy_score(y, y_pred)))
plot_confusion_matrix(bagging, X, y)

El valor de accuracy es similar, pero en este caso el modelo predice con mejor precisión las clases O y B. Sin embargo, sigue sin predecir las clases A y F.

Provemos ahora este mismo modelo eliminando los datos de las estrellas de clase A y F. En este caso el dataset está mucho más balanceado que antes. 

In [ ]:
stars_readj = stars_adj[(stars_adj[class_name] == 'M') | (stars_adj[class_name] == 'B') | (stars_adj[class_name] == 'O')]
print(Counter(stars_adj[class_name]))
hist = sns.histplot(data=stars_readj, x='class')
hist.set_title('Clases estelares en el dataset balanceadas')
plt.show()

In [ ]:
bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=2, max_leaf_nodes=2),
                            n_estimators=10,
                            max_samples=0.5, 
                            max_features=0.5)

X = stars_readj[reg_name].to_numpy()
y = stars_readj[class_name].to_numpy()

bagging.fit(X, y)

y_pred = bagging.predict(X)
print(classification_report(y, y_pred, zero_division=0))
print('accuracy score -> {0:.2%}\n\n'.format(accuracy_score(y, y_pred)))
plot_confusion_matrix(bagging, X, y)

En este caso, el modelo predice muy bien, obteniendo una accuracy del 92.65% y unos valores de F1-score cercanos a 1. Vemos que la matriz de confusión presenta los valores máximos en la diagonal, significando que cada clase se predice correctamente. 



Concluimos que balancear las clases es muy importante, porque en este último caso, las clases estaban muy bien balanceadas y se ha obtenido un resultado muy bueno. Sin embargo, se han prescindido de muchos datos y este modelo no es capaz de predecir todas las clases estelares. Concluimos también que el modelo *Bagging* obtiene mejores resultados que un modelo de clasificación simple. 

### Clustering

A continuación vamos a realizar un modelo *Hierarchical clustering* utilizando el conjunto de datos anterior. El objetivo va a ser el mismo, obtener un modelo que sea capaz de realizar una clasificación estelar en función de todos los parámetros de los que disponemos. Este modelo utiliza todo el dataset, por lo que no es necesario hacer una partición de datos de entrenamiento y validación. Utilizamos una norma euclídea con un linkage dado por la media aritmética. En este caso, la variable que contiene las clases debe ser introducida con valores numéricos. Para esto, utilizamos la clase de LabelEncoder, que convierte estas etiquetas a valores enteros, y permite transformar los datos de manera sencilla. Vamos a aplicar este modelo con los tres conjuntos de datos que hemos utilizado, `stars`, `stars_adj` y `stars_readj`. Por esta razón, definimos el modelo de una función, que devuelve el propio modelo, junto con el informe de la clasificación. 


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder

def clustering(data, n_clusters=5, linkage='ward'):
    le = LabelEncoder()
    le.fit(['B', 'M', 'O', 'A', 'F'])
    y = le.transform(data[class_name].to_numpy())
    X = data[reg_name].to_numpy()

    model = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage=linkage)
    model.fit(X, y)
    y_pred = le.inverse_transform(model.fit_predict(X))
    y = le.inverse_transform(y)
    accuracy = accuracy_score(y, y_pred)
    print('Classification report')
    print(classification_report(y, y_pred, zero_division=0))
    print('accuracy score -> {0:.2%}'.format(accuracy))
    print(30*'-')
    return model

In [ ]:
print('Dataset star')
clustering(stars)

print('\nDataset star_adj')
clustering(stars_adj)

print('\nDataset stars_readj')
clustering(stars_readj)


Vemos que todos lstars_readjos modelos devuelven unos resultados bastante malos. En los tres casos se obtiene una accuracy muy baja y unos valores de F1-scores muy por debajos del valor máximo 1. El modelo que obtiene mejor accuracy es el entrenado con el dataset completo. Sin embargo, como este dataset está muy desbalanceado, solo predice tres clases (una de ellas prácticamente no la acierta, con un F1-score de 0.06, extremadamente bajo).  
Veamos qué ocurre si modificamos el tamaño de los clusters. Como no disponemos de demasiados datos, los modelos no tardan demasiado en ajustar los parámetros, por lo que podemos entrenar todas los casos posibles, para ver con qué valor del número de clusters el modelo obtiene mejor precisión. 

In [ ]:
for n_clusters in range(1,6):
    print('Dataset star with {} clusters'.format(n_clusters))
    clustering(stars, n_clusters=n_clusters)

    print('Dataset star_adj with {} clusters'.format(n_clusters))
    clustering(stars_adj, n_clusters=n_clusters)

    print('Dataset star_readj with {} clusters'.format(n_clusters))
    clustering(stars_readj, n_clusters=n_clusters)


Vemos que el modelo que mejor precisión obtiene es el que utiliza la mayor cantidad de datos con 5 clústers. Esto era de esperar, ya que este modelo funciona mejor cuantos más datos tenga y no es necesario que las clases estén balanceadas o no. Ahora vamos a utilizar solo este conjunto de datos, modificando el linkage. 

In [ ]:
for linkage in ['ward', 'complete', 'average', 'single']:
    clustering(stars, n_clusters=5, linkage=linkage)


El mejor linkage resulta el de ward. De nuevo la accuracy sigue siendo muy baja, debido a que este modelo necesita muchos más datos para entrenar. Concluimos que para este conjunto de datos, no parece buena idea tratar de crear un modelo de clasificación basado en clusterings. 